# Setup

In [10]:
import os,time,json
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta,datetime

# KAFKA CONNECTION TO AWS

In [11]:
## let us begin by listing the buckets in our aws

In [13]:
import boto3

s3 = boto3.resource('s3')
my_bucket = s3.Bucket('10ac-batch-5')

for object_summary in my_bucket.objects.filter(Prefix="week9/"):
    print(object_summary.key)

# KAFKA

In [6]:
from kafka import KafkaConsumer,KafkaProducer,KafkaAdminClient
from kafka.admin import NewTopic

In [7]:
client = KafkaAdminClient(bootstrap_servers=['192.168.0.106:9092',
                                            'https://g4.10academy.org:9092'],
                         client_id='admin-client')

In [8]:
topic_list = []
topic_list.append(
    NewTopic(name='spark-transformed-text',
              num_partitions=1,
              replication_factor=1)
)

In [9]:
client.create_topics(new_topics=topic_list,validate_only=False)

CreateTopicsResponse_v3(throttle_time_ms=0, topic_errors=[(topic='spark-transformed-text', error_code=0, error_message=None)])

# SPARK

In [2]:
from pyspark import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
import pyspark.sql.functions as psf
from pyspark.sql.functions import lit, col,sum,avg,max,first,min,mean
import pyspark.ml as Pipeline
import pyspark.ml.pipeline as pmpip
import pyspark.ml.param as pmparam
from pyspark.ml.param.shared import HasInputCol,HasOutputCol
from pyspark.sql.types import StructType,StructField,StringType,IntegerType,FloatType

In [5]:
def create_spark_connection(**kwargs):
    app_name=kwargs.get('app_name','choquet')
    master = kwargs.get('master','yarn')
    ui_port = kwargs.get('ui_port','4044')
    driver_port = kwargs.get('driver_port','8887')
    cores_max = kwargs.get('cores_max','4')
    executor_cores = kwargs.get('executor_cores','4')
    driver_memory = kwargs.get('driver_memory','8g')
    executor_memory = kwargs.get('executor_memory','8g')
    dynamicAllocation = kwargs.get('dynamicAllocation','false')
    aqe = kwargs.get('aqe','true')
    sql_shuffle_partition = kwargs.get('sql_shuffle_partitions',200)
    
    os.environ['PYSPARK_PYTHON']='./spark_exc_env/spark_env/bin/python'
    os.environ['PYSPARK_DRIVER_PYTHON']='/usr/bin/python3'
    os.environ['SPARK_LOCAL_IP']='g4.10academy.org'
    
    conf = SparkConf()
    conf.setAppName(app_name)
    conf.set('spark.master',master)
    conf.set('spark.ui.port',ui_port)
    conf.set('spark.driver.port',driver_port)
    conf.set('spark.cores.max',cores_max)
    conf.set('spark.driver.memory',driver_memory)
    conf.set('spark.executor.memory',executor_memory)
    conf.set('spark.dynamicAllocation.enabled',dynamicAllocation)
    conf.set('spark.shuffle.service.enabled','true')
    conf.set('spark.sql.shuffle.partitions',sql_shuffle_partition)
    conf.set('spark.sql.adaptive.coalescePartitions.enabled',aqe)
    conf.set('spark.sql.parquet.compression.codec','gzip')
    conf.set('spark.executor.memoryOverhead','2g')
    conf.set('spark.shuffle.memoryFraction','1g')
    
    spark = SparkSession.builder.config(conf=conf).enableHiveSupport().getOrCreate()
    sc = spark.sparkContext()
    sc.setLogLevel('WARN')
    
    return spark,sc

def close_spark_connection(sc):
    sc.stop()
    
    

In [6]:
spark,sc=create_spark_connection(app_name='choquet')

Exception in thread "main" org.apache.spark.SparkException: When running with master 'yarn' either HADOOP_CONF_DIR or YARN_CONF_DIR must be set in the environment.
	at org.apache.spark.deploy.SparkSubmitArguments.error(SparkSubmitArguments.scala:634)
	at org.apache.spark.deploy.SparkSubmitArguments.validateSubmitArguments(SparkSubmitArguments.scala:274)
	at org.apache.spark.deploy.SparkSubmitArguments.validateArguments(SparkSubmitArguments.scala:234)
	at org.apache.spark.deploy.SparkSubmitArguments.<init>(SparkSubmitArguments.scala:119)
	at org.apache.spark.deploy.SparkSubmit$$anon$2$$anon$3.<init>(SparkSubmit.scala:1029)
	at org.apache.spark.deploy.SparkSubmit$$anon$2.parseArguments(SparkSubmit.scala:1029)
	at org.apache.spark.deploy.SparkSubmit.doSubmit(SparkSubmit.scala:85)
	at org.apache.spark.deploy.SparkSubmit$$anon$2.doSubmit(SparkSubmit.scala:1046)
	at org.apache.spark.deploy.SparkSubmit$.main(SparkSubmit.scala:1055)
	at org.apache.spark.deploy.SparkSubmit.main(SparkSubmit.scal

RuntimeError: Java gateway process exited before sending its port number

In [7]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession, Catalog

spark_conf = SparkConf()
spark_conf.setAll([
    ('spark.master', 'spark://g4.10academy.org:80'),
    ('spark.app.name', 'myApp'),
    ('spark.submit.deployMode', 'client'),
    ('spark.ui.showConsoleProgress', 'true'),
    ('spark.eventLog.enabled', 'false'),
    ('spark.logConf', 'false'),
    ('spark.driver.bindAddress', '0.0.0.0'),
    ('spark.driver.host', 'g4.10academy.org')
])

spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/09 19:24:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4046. Attempting port 4047.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4047. Attempting port 4048.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4048. Attempting port 4049.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4049. Attempting port 4050.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on port 4051. Attempting port 4052.
22/07/09 19:24:11 WARN Utils: Service 'SparkUI' could not bind on

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/martin/miniconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/martin/miniconda3/lib/python3.9/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/martin/miniconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/martin/miniconda3/lib/python3.9/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer 

Py4JError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext

In [4]:
spark = SparkSession.builder.appName('testing').getOrCreate()

22/07/08 15:37:12 WARN Utils: Your hostname, martin-HP-EliteBook-x360-1030-G2 resolves to a loopback address: 127.0.1.1; using 192.168.0.106 instead (on interface wlp58s0)
22/07/08 15:37:12 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/08 15:37:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df_pyspark = spark.read.csv('../data/Amharic News Dataset.csv',header=True,inferSchema=True)

In [6]:
df_pyspark.show()

+--------------------+--------------------+-----------------+-----+--------------------+--------------------+
|            headline|            category|             date|views|             article|                link|
+--------------------+--------------------+-----------------+-----+--------------------+--------------------+
|የኦሊምፒክ ማጣሪያ ተሳታፊዎ...|                ስፖርት| January 14, 2021|    2|ብርሃን ፈይሳየኢትዮጵያ ቦክ...|https://www.press...|
|          አዲስ ዘመን ድሮ|                መዝናኛ|December 28, 2020|    4| የአዲስ ዘመን ጋዜጣ ቀደም...|                null|
|      መንገድ በመከተል አልፎ|                null|             null| null|                null|                null|
|      አልፎ በሚገኙት ሰፈሮች|                null|             null| null|                null|                null|
|       ብዙዎች የልኳንዳ ሥጋ|                null|             null| null|                null|                null|
|የተሰቀለባቸውና   ፉርኖ ይ...|https://www.press...|             null| null|                null|                null|
|የአረንጓዴ ጎር